In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [4]:
#Variable 
#var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
var ="solidprcptot"
#var ="prcptot"
#var ="dlyfrzthw"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_monthly.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_monthly.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+var+"_monthly.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_solidprcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM

In [5]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_solidprcptot_monthly.nc']

# FOR RCP45 

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.month[0:12]
# Monthly 30 y means
listds45 = []
i=0
for s in dsEns45.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds45.append(tmp1)

0
<xarray.Dataset>
Dimensions:       (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * time          (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lat           (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon           (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:       (lat: 320, lo

In [7]:
i = 0
df45list = []
for ds in listds45:
    print ("Dataset number: ", i)
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df45=0
    df453 = df452.loc[df452["realization"] == 0].dropna()
    df452 = 0
    print ("Converted to DF")
    dfp452 =  df453.reset_index()
    df453 =0
    df45C = dfp452.copy()
    dfp452=0
    df45C[var+"_p10"] = round(df45C[var+"_p10"],2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"],2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    df45C=0
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"  
    Region1i45["month"] = month
    df45list.append(Region1i45)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  4
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  5
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  6
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  7
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  8
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF a

In [8]:
listds45 =0
dsEns45=0

In [9]:
df45 = pd.concat(df45list)

In [10]:
df45.reset_index().to_feather("/home/mlopez/EXEC/Processed data to clip with regions/df45"+var+"_forTS_monthly.feather")

In [11]:
df45 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/df45"+var+"_forTS_monthly.feather")

In [12]:
df45list =0

# FOR RCP85 

In [13]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.month[0:12]
# Monthly 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:       (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * time          (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lat           (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon           (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:       (lat: 320, lo

In [14]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df85=0
    df853 = df852.loc[df852["realization"] == 0].dropna()
    df852=0
    print ("Converted to DF")
    dfp852 =  df853.reset_index()
    df853=0
    df85C = dfp852.copy()
    df85C[var+"_p10"] = round(df85C[var+"_p10"],2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"],2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    df85C=0
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"
    Region1i85["month"] = month
    df85list.append(Region1i85)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  4
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  5
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  6
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  7
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  8
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF a

In [15]:
listds85=0
dsEns85=0

In [16]:
df85 = pd.concat(df85list)

In [17]:
df85.reset_index().to_feather("/home/mlopez/EXEC/Processed data to clip with regions/df85"+var+"_forTS_monthly.feather")

In [18]:
df85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/df85"+var+"_forTS_monthly.feather")

In [19]:
df85list=0

# OBSERVED DATA

In [20]:
dsOBs= xr.open_mfdataset(filesobs)

will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  dsOBs= xr.open_mfdataset(filesobs)

to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  combined = auto_combine(



In [21]:
dsOBs

<xarray.Dataset>
Dimensions:       (lat: 510, lon: 1068, time: 768)
Coordinates:
  * time          (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2013-12-01
  * lon           (lon) float32 -140.95833 -140.875 ... -52.125004 -52.04167
  * lat           (lat) float32 83.45833 83.375 83.291664 ... 41.125 41.041668
Data variables:
    solidprcptot  (time, lat, lon) float32 dask.array<chunksize=(768, 510, 1068), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           NRCAN 10km Gridded Climate Dataset
    history:         2012-10-19T16:38:49: Convert from original format to NetCDF
    institution:     NRCAN
    source:          ANUSPLIN
    redistribution:  Redistribution policy unknown. For internal use only.

## Convert in Dataframe

In [22]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0
dsOBs=0

In [23]:
dfObs = dfmeanObs.rename(columns={var: var+'_Obs'})
dfmeanObs=0

In [24]:
month_dict = {1:"january", 2:"february", 3:"march", 4:"april", 5:"may", 6:"june", 7:"july", 
              8:"august", 9:"september", 10:"october", 11:"november", 12:"december" }

dfObs["month"]= dfObs.time.dt.month.map(lambda m: month_dict[m])
dfObs

,lat,lon,time,solidprcptot_Obs,month
0,83.125000,-74.541672,1950-01-01,6.63,january
1,83.125000,-74.541672,1950-02-01,8.13,february
2,83.125000,-74.541672,1950-03-01,3.47,march
3,83.125000,-74.541672,1950-04-01,3.56,april
4,83.125000,-74.541672,1950-05-01,9.79,may
...,...,...,...,...,...
197577979,41.708332,-82.208336,2013-08-01,0.00,august
197577980,41.708332,-82.208336,2013-09-01,0.00,september
197577981,41.708332,-82.208336,2013-10-01,0.00,october
197577982,41.708332,-82.208336,2013-11-01,3.46,november


In [25]:
dfObs.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

In [26]:
dfObs = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

## Merge 2 rcp dataframes

In [27]:
df2m =  pd.merge(df45, df85, on=["lat","lon", "time", "month"], suffixes=('_45', '_85'))
df45 = 0
df85 = 0

In [28]:
df2m

,level_0_45,index_45,lat,lon,time,solidprcptot_p10_45,solidprcptot_p50_45,solidprcptot_p90_45,month,level_0_85,index_85,solidprcptot_p10_85,solidprcptot_p50_85,solidprcptot_p90_85
0,0,0,66.623306,-89.045212,1950-01-01,3.76,10.17,16.33,january,0,0,3.57,10.03,16.24
1,1,1,66.623306,-89.045212,1951-01-01,4.82,8.28,15.35,january,1,1,4.73,8.34,15.25
2,2,2,66.623306,-89.045212,1952-01-01,4.40,11.07,14.83,january,2,2,4.42,11.29,14.84
3,3,3,66.623306,-89.045212,1953-01-01,4.91,8.64,15.27,january,3,3,4.70,8.61,15.45
4,4,4,66.623306,-89.045212,1954-01-01,4.06,10.69,15.27,january,4,4,3.99,10.80,15.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138085267,11507101,220720470,40.041039,-74.045807,2096-12-01,0.00,0.00,7.58,december,11507101,220720470,0.00,0.00,1.68
138085268,11507102,220720471,40.041039,-74.045807,2097-12-01,0.00,2.68,13.17,december,11507102,220720471,0.00,0.00,4.35
138085269,11507103,220720472,40.041039,-74.045807,2098-12-01,0.20,1.85,25.34,december,11507103,220720472,0.00,0.00,0.56
138085270,11507104,220720473,40.041039,-74.045807,2099-12-01,0.00,0.57,4.02,december,11507104,220720473,0.00,0.00,2.30


In [29]:
df2m.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/df2m"+var+"_forTS_monthly.feather")

In [30]:
df2m = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/df2m"+var+"_forTS_monthly.feather")

# Cut by region for rcp45, 85 and Obs

## Get the lat lon regions

In [31]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [32]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

['/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_REG_FOR_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_UA_PER_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/REG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SDOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/Secteurs_Operations_Regionaleslat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SREG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/territoire_guidelat_lon.feather']

In [33]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

## Create DF by region

In [34]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [35]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [36]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

## Cutting rcp45 and 85 by region and getting df with 4th degree polynomial

In [37]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [38]:
monthlist = ["january", "february", "march", "april", "may", "june", "july", "august", "september", 
             "october", "november", "december"]

In [ ]:
for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/8... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/8... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, df2m, on=["lat","lon"])
    dfdataregObs = pd.merge(dfregion, dfObs, on=["lat","lon"])
    for mon in monthlist:
        print (mon)
        print ("3/8... Get mean by year for every sub-region for every month")
        dfmeanUA45 = dfdatareg45.loc[dfdatareg45['month'] == mon]
        dfmean45UA = dfmeanUA45.groupby(["time", name, 'month']).mean()
        dfmean45UA2 = dfmean45UA.reset_index()
        dfmean45UA=0
        dfmeanObs = dfdataregObs.loc[dfdataregObs['month'] == mon]
        dfmeanObs2 = dfmeanObs.groupby(["time", name, 'month']).mean() 
        dfmeanObs =0
        dfmeanObs3 = dfmeanObs2.reset_index()
        dfmeanObs2=0
        print("Merging 3 cases")
        df3m = pd.merge(dfmean45UA2, dfmeanObs3, how='left', on= ["time", name, "month"] )
        dfmeanObs3=0
        dfmean45UA2=0
        print ("4/8... Split by subregion")
        df3ma = df3m[['time', name, var+'_p10_45', var+'_p50_45', var+'_p90_45',
                                   var+'_p10_85', var+'_p50_85', var+'_p90_85', var+'_Obs', "month"]]
        df3m=0
        if (var == "tg_mean" or var== "tn_mean" or var=="tx_mean"):
            df3ma[var+"_Obs"] = df3ma[var+"_Obs"]- 273.15
        if (var == "solidprcptot"):
            df3ma[var+"_Obs"] = df3ma[var+"_Obs"]/10
        df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)
        for name2,dfsr in df3ma.groupby(name):
            print ("                   "+name2, name)
            dfsrc = dfsr.copy()
            print ("6/8... 4th Polynomial - rcp45,85")
            years = dfsrc.time.dt.year
            dfsrc["time"] = years#.values
            for p in ('_p10_45', '_p50_45', '_p90_45', '_p10_85', '_p50_85', '_p90_85'):
                if (var == "tg_mean" or var== "tn_mean" or var=="tx_mean"):
                    dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4) - 273.15
                if (var == "solidprcptot"):
                    dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4)/10
                if (var == "prcptot", "dlyfrzthw", "DJC"):
                    dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4)
                dfsrc[var+p] = round(dfsrc[var+p], 2) 
            print(" -----------------------8/8 Creating CSV file for sub-region " +name2)
            dfsrc.to_csv(replace_all(name2,d)+"_"+var+"_"+mon+".csv")


------------------------------------------------------------------
1/8... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/8... Merging region with data
january
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   ABITIBI-TEMISCAMINGUE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region ABITIBI-TEMISCAMINGUE
                   BAS-SAINT-LAURENT NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region BAS-SAINT-LAURENT
                   CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
                   COTE-NORD NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region COTE-NORD
                   ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL 

                   OUTAOUAIS NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region OUTAOUAIS
                   SAGUENAY -LAC-SAINT-JEAN NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region SAGUENAY -LAC-SAINT-JEAN
may
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   ABITIBI-TEMISCAMINGUE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region ABITIBI-TEMISCAMINGUE
                   BAS-SAINT-LAURENT NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region BAS-SAINT-LAURENT
                   CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
                   COTE-NORD NM_REG_FOR
6/8

                   NORD-DU-QUEBEC NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region NORD-DU-QUEBEC
                   OUTAOUAIS NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region OUTAOUAIS
                   SAGUENAY -LAC-SAINT-JEAN NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region SAGUENAY -LAC-SAINT-JEAN
september
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   ABITIBI-TEMISCAMINGUE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region ABITIBI-TEMISCAMINGUE
                   BAS-SAINT-LAURENT NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region BAS-SAINT-LAURENT
                   CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES NM_REG_FOR
6/8... 4th Polyno

 -----------------------8/8 Creating CSV file for sub-region SAGUENAY -LAC-SAINT-JEAN
------------------------------------------------------------------
1/8... Cutting by region
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA
2/8... Merging region with data
january
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   01171 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 01171
                   01272 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 01272
                   02371 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02371
                   02471 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02471
                   02571 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating

Merging 3 cases
4/8... Split by subregion
                   01171 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 01171
                   01272 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 01272
                   02371 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02371
                   02471 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02471
                   02571 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02571
                   02651 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02651
                   02661 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02661
             

...

                   11262 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 11262
                   11263 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 11263
november
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   01171 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 01171
                   01272 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 01272
                   02371 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02371
                   02471 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02471
                   02571 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 ---------

 -----------------------8/8 Creating CSV file for sub-region 11263
december
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   01171 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 01171
                   01272 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 01272
                   02371 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02371
                   02471 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02471
                   02571 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02571
                   02651 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 02651
           

2/8... Merging region with data
january
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   Golfe du Saint-Laurent NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Golfe du Saint-Laurent
                   Pessière à lichens NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Pessière à lichens
                   Pessière à mousses NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Pessière à mousses
                   Sapinière à bouleau blanc NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Sapinière à bouleau blanc
                   Sapinière à bouleau jaune NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Sapinière à bouleau jaune
                   Toundra arctique arbustive 

                   Toundra arctique herbacée NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Toundra arctique herbacée
                   Toundra forestière NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Toundra forestière
                   Érablière à bouleau jaune NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Érablière à bouleau jaune
                   Érablière à caryer cordiforme NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Érablière à caryer cordiforme
                   Érablière à tilleul NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Érablière à tilleul
june
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   Golfe du Saint-Laurent NOM
6/8... 4th Polynomi

 -----------------------8/8 Creating CSV file for sub-region Érablière à tilleul
october
3/8... Get mean by year for every sub-region for every month
Merging 3 cases
4/8... Split by subregion
                   Golfe du Saint-Laurent NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Golfe du Saint-Laurent
                   Pessière à lichens NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Pessière à lichens
                   Pessière à mousses NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Pessière à mousses
                   Sapinière à bouleau blanc NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Sapinière à bouleau blanc
                   Sapinière à bouleau jaune NOM
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region Sapinière à bouleau jau

In [ ]:
df2m =0

In [ ]:
dfregion=0
dfObs=0